# KI + SPARQL suche

Wie in dem letzten Beispiel angedeutet können wir natürlich versuchen, Schlagwörter zu einem Objekt (hier ein Bild) mit Hilfe von KI und kontrollierten Vokabular zu vergeben.

In dem Beispiel unten verwenden wir weiterhin ein Bild von IIIF-Manifest. Wir lassen GPT-4.1 von OpenAI die Schlagwörter herausfinden. Dabei soll das KI-Modell die Schlagwörter aus GND suchen.

In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import requests
import html
import re
import json


In [2]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
# Tools definieren für function calling (Schnittstelle für GPT-4.1)
tools = [{
  "type": "function",
  "name": "gnd_search",
  "description": "Suche nach GND-Schlagwwörter.",
  "parameters": {
      "type": "object",
      "properties": {
        "query_text": {"type": "string", "description": "Suche nach passenden Schlagwörtern auf Deutsch in GND. Teilweise Übereinstimmung zugelassen"},
        "limit": {"type": "integer", "default": 100}
      },
      "required": ["query_text"]
        }
    },
    {
        "type": "function",
        "name": "gnd_person_search",
        "description": "Suche eine Person in GND.",
        "parameters": {
            "type": "object",
            "properties": {
                "query_text": {"type": "string", "description": "Personenname auf Deutsch. Z.B. Rembrandt, van Rijn"},
                "limit": {"type": "integer", "default": 30},
                "use_regex": {"type": "boolean", "default": False, "description": "Ob man Regex verwendet, oder nicht"}
            },
            "required": ["query_text"]
        }
    }
]

In [8]:
def _escape_for_sparql(text: str) -> str:
    return text.replace("\\", "\\\\").replace('"', '\\"')

def gnd_search_impl(query_text: str, limit: int = 100):
    
    q = _escape_for_sparql(query_text.strip())
    sparql = f"""
PREFIX gndo: <https://d-nb.info/standards/elementset/gnd#>
SELECT ?subject ?label ?broader ?broaderLabel ?related ?relatedLabel WHERE {{
  ?subject gndo:preferredNameForTheSubjectHeading | gndo:variantNameForTheSubjectHeading ?label .
  FILTER regex (?label, LCASE("{q}"), "i")  
  
  OPTIONAL {{
    ?subject gndo:broaderTermGeneral ?broader .
    ?broader gndo:preferredNameForTheSubjectHeading ?broaderLabel .
    
  }}
  OPTIONAL {{
    ?subject gndo:relatedTerm ?related .
    ?related gndo:preferredNameForTheSubjectHeading ?relatedLabel .
    
  }}
}}
LIMIT {int(limit)}
"""
    r = requests.get(
        "https://sparql.dnb.de/api/gnd",
        params={"query": sparql, "action": "sparql", "format": "application/sparql-results+json"},
        timeout=30,
    )
    r.raise_for_status()
    data = r.json()
    rows = []
    for b in data.get("results", {}).get("bindings", []):
        def v(obj, key): return obj.get(key, {}).get("value")
        rows.append({
            "subject": v(b, "subject"),             
            "label": v(b, "label"),                 
            "broader": v(b, "broader"),
            "broaderLabel": v(b, "broaderLabel"),
            "related": v(b, "related"),
            "relatedLabel": v(b, "relatedLabel"),
        })
    # Verdopplung weghauen
    seen = set()
    uniq = []
    for r2 in rows:
        if r2["subject"] not in seen:
            seen.add(r2["subject"])
            uniq.append(r2)
    return {"hits": uniq}


In [10]:
def _escape_for_sparql_for_person(text: str) -> str:
    return text.replace("\\", "\\\\").replace('"', '\\"')

def gnd_person_search_impl(query_text: str, limit: int = 20, use_regex: bool = False):
    q = query_text.strip()
    if use_regex:
        filter_clause = f'FILTER regex (?label, "{_escape_for_sparql_for_person(q)}", "i")'
    else:
        filter_clause = f'FILTER(CONTAINS(LCASE(STR(?label)), LCASE("{_escape_for_sparql_for_person(q)}")))'

    sparql = f"""
PREFIX gndo: <https://d-nb.info/standards/elementset/gnd#>
SELECT ?subject ?label ?pref ?variant ?birth ?death ?occupation ?occLabel WHERE {{
  {{
    ?subject gndo:preferredNameForThePerson ?label .
    BIND(?label AS ?pref)
  }} UNION {{
    ?subject gndo:variantNameForThePerson ?label .
  }}
  {filter_clause}  
}}
LIMIT {int(limit)}
"""

    
    r = requests.get(
        "https://sparql.dnb.de/api/gnd",
        params={"query": sparql, "action": "sparql", "format": "application/sparql-results+json"},
        timeout=30,
    )
    r.raise_for_status()
    data = r.json()

    rows = []
    for b in data.get("results", {}).get("bindings", []):
        def v(key): return b.get(key, {}).get("value")
        rows.append({
            "subject": v("subject"),          
            "label": v("label"),              
            "preferred": v("pref"),           
            "variant": v("variant"),         
            "birth": v("birth"),
            "death": v("death"),
            "occupation": v("occupation"),
            "occupationLabel": v("occLabel"),
        })

    
    seen, uniq = set(), []
    for r2 in rows:
        sid = r2["subject"]
        if sid not in seen:
            seen.add(sid)
            uniq.append(r2)
    return {"hits": uniq}


In [11]:
iiif_manifest_link = "https://api.artic.edu/api/v1/artworks/49047/manifest.json"
iiif_manifest = requests.get(iiif_manifest_link)

manifest = iiif_manifest.json()
work_info = {}
metadata = manifest["metadata"]

for i in metadata:
    if i["label"] == "Artist / Maker":
        work_info["creator"] = i["value"]
    else:
        continue



seq = manifest["sequences"][0]
canvases = seq["canvases"]

works = []

for j in canvases:
    work = {}
    work["title"] = j["label"]
    work["image_url"] = j["images"][0]["resource"]["@id"]
    works.append(work)

work_info["works"] = works

user_question = f"Das ist ein Werk von {work_info['creator']}. Der Titel des Werkes lautet {work_info['works'][0]['title']}."

In [20]:
system_hint = (
  "Du arbeitest mit GND (deutsche LOD). "
  "Extrahiere nur deutsche Kandidatenbegriffe. "
  "Wenn es sich um eine Person handelt (z.B. Signatur, Porträt, Künstlername), rufe gnd_person_search auf; "
  "für Sachthemen rufe gnd_search auf. "
  "Bei beiden Suchen setze ein grosszügiges Limit, mindestens 30."
  "Bei der Suche nach Perosnennamen verwende die Namensform, '(Nachnachme), (Vorname)'."
  "Du kannst Gattung des Kunstwerkes (z.B. Porträt, Landschaftsmalerei) mit gnd_search suchen."
)

In [21]:
resp = client.responses.create(
  model="gpt-4.1",
  input=[{
    "role": "user",
    "content": [
      {"type":"input_text","text": system_hint + user_question},
      {"type":"input_image","image_url": work_info["works"][0]["image_url"] }
    ]
  }],
  tools=tools,
  tool_choice="auto",
)


In [22]:
print(resp)

Response(id='resp_043f59e7d2ccd8f80068e51d3819c48190942f288bf3bcdb9d', created_at=1759845688.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFunctionToolCall(arguments='{"query_text":"Rembrandt, van Rijn","limit":30,"use_regex":false}', call_id='call_lgE3MPMl1BFBx3EiBWISenSs', name='gnd_person_search', type='function_call', id='fc_043f59e7d2ccd8f80068e51d3bd47c81908b71ba67776e67dd', status='completed'), ResponseFunctionToolCall(arguments='{"query_text":"Bauer","limit":30}', call_id='call_wCmbZ6UuXsmM2j7HST1sy4ZB', name='gnd_search', type='function_call', id='fc_043f59e7d2ccd8f80068e51d3c5aa08190941aacfed3161867', status='completed'), ResponseFunctionToolCall(arguments='{"query_text":"Porträt","limit":30}', call_id='call_pENz6n4JVROhEgvz4tMtpcha', name='gnd_search', type='function_call', id='fc_043f59e7d2ccd8f80068e51d3ccf748190a7be7ec246a6e56d', status='completed'), ResponseFunctionToolCall(arguments

In [23]:


tool_calls = [x for x in resp.output if getattr(x, "type", "") == "function_call"]

messages = []
for call in tool_calls:
    args = json.loads(call.arguments) if isinstance(call.arguments, str) else (call.arguments or {})

    if call.name == "gnd_search":
        result = gnd_search_impl(**args)

    elif call.name == "gnd_person_search":
        result = gnd_person_search_impl(**args)

    else:
        result = {"error": f"unknown tool {call.name}"}

    messages.append({
        "type": "function_call_output",
        "call_id": call.call_id,
        "output": json.dumps(result, ensure_ascii=False)
    })

follow = client.responses.create(
    model="gpt-4.1",
    previous_response_id=resp.id,
    input=messages,
    tools=tools,   
)


print(follow.output_text or "(no output_text)")


Hier sind die passenden deutschen Kandidatenbegriffe aus der GND für das beschriebene Werk:

1. Person:
   - Rembrandt, van Rijn (https://d-nb.info/gnd/11859964X)

2. Sachthemen / Gattungen:
   - Radierung (https://d-nb.info/gnd/4048166-9) (technisch: Druckgrafikverfahren)
   - Bauer (kein Treffer als Personengruppe/Sachbegriff; nur viele zusammengesetzte Begriffe, daher am ehesten: "Bauer" als Darstellung)
   - Porträt (kein Sachbegriff gefunden, aber für Kunstgattung relevant)

Empfohlene GND-Begriffe für die Verschlagwortung:
- Rembrandt, van Rijn (Person)
- Radierung (Gattung/Technik)
- Bauer (Dargestellte Person/Typus)
- Porträt (Gattung, falls gebraucht – kein GND-Eintrag gefunden, aber in der allgemeinen Kunstterminologie üblich)

GND-Links:
- Rembrandt, van Rijn: https://d-nb.info/gnd/11859964X
- Radierung: https://d-nb.info/gnd/4048166-9

Weitere spezifische Sachbegriffe wie "Peasant" (= Bauer) oder "Porträt" ergeben in der GND-Suche aktuell keine eindeutigen Einträge als Kuns